In [3]:
#Main model
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
main_model = load_model('Model\model.h5')

In [11]:
mixer.init()
alarm=mixer.Sound('alarm.wav')
capture = cv2.VideoCapture(0)
score = 0 #since counting are fast like in miliseconds.
while True:
    ret,frame = capture.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),color= (0,0,255),thickness=3)
    
    for (x,y,w,h) in eyes:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),color= (0,0,255),thickness=3)
        #preprocessing step
        eye = frame[y:y+h,x:x+w]
        eye = cv2.resize(eye,(80,80)) #resize image
        eye = eye/255 #rescaling image by 255
        eye = eye.reshape(80,80,3) #reshape
        eye = np.expand_dims(eye,axis=0)
        #model prediction here
        prediction = main_model.predict(eye)
        if prediction[0][0] > 0.30: #if eyes are close
            score = score + 1
            if score>10:
                try:
                    alarm.play()
                except:
                    pass
        elif prediction[0][1] > 0.90:
            score = score - 1
            if score <0:
                score = 0
 
        # print(prediction)
    cv2.imshow('Drowsiness Detection',frame)
    if cv2.waitKey(35) & 0xFF==ord('Q'):
        break
capture.release()
cv2.destroyAllWindows()
